In [5]:
from sentinelhub import (
    CRS,
    BBox,
    BBoxSplitter,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubCatalog,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    TileSplitter,
    read_data,
    SHConfig
)



import os
import numpy as np
import math
import matplotlib.pyplot as plt
import xarray as xr  # It may need Dask library https://docs.dask.org/en/stable/install.html
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr  # It may need Dask library https://docs.dask.org/en/stable/install.html
import re

from evalScripts_download import  NDVI

from mgrs import MGRS
from folium import GeoJson
from folium.plugins import MarkerCluster
from datetime import datetime, timedelta
from shapely.geometry import MultiLineString, MultiPolygon, Polygon, shape
from matplotlib.patches import Polygon as PltPolygon
from mpl_toolkits.basemap import Basemap 
from dotenv import load_dotenv

In [4]:
import numpy as np
import rasterio as rio
from rasterio import features
from rasterio.merge import merge
from rasterio.plot import show
import matplotlib.pyplot as plt


from shapely.geometry import shape, mapping, box
import geopandas as gpd
from pyproj import Transformer

In [6]:
load_dotenv()
# authenticate to SH using client and client secret
config = SHConfig()

config.sh_client_id = os.getenv("SH_CLIENT_ID")
config.sh_client_secret = os.getenv("SH_CLIENT_SECRET")

catalog = SentinelHubCatalog(config=config)

In [7]:
input_file_path = r"geometry/aoi_3.json"
geojson = read_data(input_file_path)
aoi = shape(geojson["features"][0]["geometry"])

resolution = 10  # meters
aoi_bbox = aoi.bounds
aoi_bbox = BBox(bbox=aoi_bbox, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
aoi_size

(2030, 2378)

In [8]:
def calculateGridSize(bbox_size):
    width, height = bbox_size

    # Calculate the number of tiles needed for width and height
    tiles_x = math.ceil(width / 1200)
    tiles_y = math.ceil(height / 1200)

    return (tiles_x, tiles_y)

In [9]:
gridsize = calculateGridSize(aoi_size)
bbox_splitter = BBoxSplitter([aoi], CRS.WGS84, gridsize, reduce_bbox_sizes=True)
gridsize

(2, 2)

In [10]:
output_folder = r"results"
if not os.path.exists(f"{output_folder}"):
    os.makedirs(f"{output_folder}")

In [11]:
start_date = "2024-08-01"
end_date = "2024-08-15" 

In [12]:
def get_request_by_subarea(bbox, evalscript, start_date, end_date):
    size = bbox_to_dimensions(bbox, resolution=resolution)
    return SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(start_date, end_date),
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        data_folder=f"{output_folder}",
        config=config,
    )

In [13]:
bbox_list = bbox_splitter.get_bbox_list()
sh_requests_deforestation = [
    get_request_by_subarea(bbox, NDVI, start_date, end_date)
    for bbox in bbox_list
]
dl_requests = [request.download_list[0] for request in sh_requests_deforestation]

# download data with multiple threads
downloaded_data = SentinelHubDownloadClient(config=config).download(
    dl_requests, max_threads=5
)

In [14]:
import rasterio as rio

In [87]:
image_path = "results/9c7c8fad062a93139dd32cae7c9fd115/response.tiff"

In [114]:
tiffiles = []
for folder, subfolder, files in os.walk(output_folder):
    for file in files:
        if file.lower().endswith((".tif",".tiff")):
            tiffiles.append(os.path.join(folder, file))

In [115]:
tiffiles

['results\\mosaico.tif',
 'results\\1660e055e6230a1b4f0235a5458abbed\\response.tiff',
 'results\\19d850aedd1cefe1563a1761eda8f78b\\response.tiff',
 'results\\88a349c5fd83ad6d3bd92123cc23dc44\\response.tiff',
 'results\\9c7c8fad062a93139dd32cae7c9fd115\\response.tiff']

In [116]:
output_folder

'results'

In [ ]:
mosaicFiles =  [rio.open(file) for file in tiffiles]

# Hacer el mosaico
mosaic, out_trans = merge(mosaicFiles)

# Copiar metadatos del primero
out_meta = mosaicFiles[0].meta.copy()

# Actualizar metadatos con nueva dimensión y transform
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans
})

# Guardar mosaico a disco
out_fp = "mosaico.tif"
with rio.open(os.path.join(output_folder), "w", **out_meta) as dest:
    dest.write(mosaic)

CPLE_AppDefinedError: Deleting results\mosaico.tif failed: Permission denied

In [ ]:
with rio.open(os.path.join(output_folder,"mosaico.tif")) as src:
    raster_data = src.read(1)  # primera banda
    transform = src.transform
    crs = src.crs
    width, height = src.width, src.height
    bounds = src.bounds

In [ ]:
indice = "NDVI"

In [ ]:
combined_state = {
    "ndvi": {
        1: ["Ausencia de vegetación o cobertura vegetal muy escasa", "< 0"],
        2: ["Vegetación escasa o estresada", "0 - 0.2"],
        3: ["Vegetación en condiciones moderadas", "0.2 - 0.5"],
        4: ["Vegetación densa y saludable", "0.5 - 0.8"],
        5: ["Vegetación extremadamente densa y saludable", "0.8 - 1"],
    },
    "gndvi": {
        1: ["Áreas con baja o nula vegetación, suelos desnudos, agua o superficies construidas", "< 0"],
        2: ["Vegetación en condiciones saludables, pero no particularmente densa", "0 - 0.5"],
        3: ["Vegetación densa y saludable", "0.5 - 0.8"],
        4: ["Vegetación extremadamente densa y saludable", "0.8 - 1"]
    },
    "evi": {
        1: ["Zonas con muy poca o ninguna vegetacion", "< 0"],
        2: ["Presencia creciente de vegetacion", "0 - 0.5"],
        3: ["Vegetacion mas densa y saludable que las areas verdes", "0.5 - 0.8"],
        4: ["Vegetacion con la maxima densidad y salud de la vegetación", "0.8 - 1"],
    },
    "savi": {
        1: ["Muy baja vegetacion o suelo expuesto", "< 0"],
        2: ["Vegetación escasa", "0 - 0.5"],
        3: ["Vegetación moderada", "0.5 - 0.8"],
        4: ["Vegetación extremadamente densa", "0.8 - 1"]
    },
    "ndci": {
        1: ["Vegetacion con baja concentacion de clorofila", "< 0"],
        2: ["Vegetación con concentracion de clorofila moderada", "0 - 0.2"],
        3: ["Vegetacion con alta concentacion de clorofila", "0.2 - 1"],
    },
    "ndmi": {
        1: ["Baja humedad en la vegetación", "< 0"],
        2: ["Moderada humedad en la vegetacion", "0 - 0.5"],
        3: ["Alta humedad en la vegetación", "0.5 - 0.8"],
        4: ["Muy alta humedad en la vegetación", "0.8 - 1"],
    },
    "ndsi": {
        1: ["Zonas Ausencia de Nieve", "< 0"],
        2: ["Zonas con poca nieve", "0 - 0.4"],
        3: ["Zonas con nieve densa", "0.4 - 1"]
    },
    "ndwi": {
        1: ["Zonas sin presencia de agua", "< 0"],
        2: ["Zonas inundadas", "0 - 1"],
    },
    "ndbi": {
        1: ["Baja presencia de superficies construidas o alta presencia de vegetación", "< -0.05"],
        2: ["Zonas construidas con vegetación", "-0.05 - 0.05"],
        3: ["Zonas con alta construccion", "0.05 - 1"],
    },
    "nbr": {
        1: ["Zonas no quemadas", "< 0.1"],
        2: ["Zonas quemadas con gravedad moderada baja", "0.1 - 0.44"],
        3: ["Zonas quemadas con gravedad moderada alta", "0.44 - 0.66"],
        4: ["Zonas gravemente quemadas", "0.66 - 1"]
    }
}

def getClassification(index: str):
    """
    Devuelve la clasificación de estados para el índice solicitado.
    """
    return combined_state.get(index.lower(), combined_state["nbr"])

In [ ]:
reclasification_methods = {
        "ndvi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.2, 2,
                                                     np.where(x <= 0.5, 3,
                                                              np.where(x <= 0.8, 4, 5))))),
        "gndvi": lambda x: np.where(np.isnan(x), np.nan,
                                    np.where(x < 0, 1,
                                             np.where(x <= 0.5, 2,
                                                      np.where(x <= 0.8, 3, 4)))),
        "evi": lambda x: np.where(np.isnan(x), np.nan,
                                  np.where(x < 0, 1,
                                           np.where(x <= 0.5, 2,
                                                    np.where(x <= 0.8, 3, 4)))),
        "savi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.5, 2,
                                                     np.where(x <= 0.8, 3, 4)))),
        "ndci": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.2, 2, 3))),
        "ndmi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.5, 2,
                                                     np.where(x <= 0.8, 3, 4)))),
        "ndsi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1,
                                            np.where(x <= 0.45, 2, 3))),
        "ndwi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x < 0, 1, 2)),
        "ndbi": lambda x: np.where(np.isnan(x), np.nan,
                                   np.where(x <= -0.05, 1,
                                            np.where(x <= 0.05, 2, 3))),
        "nbr": lambda x: np.where(np.isnan(x), np.nan,
                                  np.where(x <= 0.1, 1,
                                           np.where(x <= 0.44, 2,
                                                    np.where(x <= 0.66, 3, 4)))),
    }

In [ ]:
method = reclasification_methods[indice.lower()]
reclasificado = method(raster_data)

In [ ]:
unique_vals = np.unique(reclasificado[~np.isnan(reclasificado)])
IndexGroups = {int(val): [] for val in unique_vals}

In [ ]:
for val in unique_vals:
        vals = raster_data[reclasificado == val]
        IndexGroups[int(val)] = vals.tolist()

In [ ]:
mask = ~np.isnan(reclasificado)

In [ ]:
polygons = []
for geom, value in features.shapes(reclasificado.astype(np.int16),
                                mask=mask,
                                transform=transform):
    if value == 0:
        continue
    polygons.append({"geometry": shape(geom), "value": int(value)})

gdf = gpd.GeoDataFrame(polygons, crs=crs)

In [ ]:
dissolved = gdf.dissolve(by="value", as_index=False)

NameError: name 'stateClasification' is not defined

In [3]:
stateClasification = getClassification(indice)

NameError: name 'getClassification' is not defined

In [ ]:


dissolved["IndexState"] = dissolved["value"].apply(
    lambda v: stateClasification.get(int(v), ["Sin clasificacion"])[0]
)
dissolved["IndexRange"] = dissolved["value"].apply(
    lambda v: stateClasification.get(int(v), ["", "Sin clasificacion"])[1]
)
dissolved["IndexRangeMean"] = dissolved["value"].apply(
    lambda v: round(sum(IndexGroups.get(int(v), [])) / len(IndexGroups.get(int(v), [])), 4)
    if IndexGroups.get(int(v)) else None
)

In [2]:
stateClasification.get(int(v), ["", "Sin clasificacion"])[1]

NameError: name 'stateClasification' is not defined

In [ ]:
dissolved = dissolved.to_crs(epsg=4326)

In [ ]:
dissolved.to_file(os.path.join(output_folder,"CLASIFICADO.geojson"), driver="GeoJSON")